In [1]:
!pip install torch
!pip install transformers

     |████████████████████████████████| 2.0MB 5.3MB/s 
     |████████████████████████████████| 890kB 26.3MB/s 
     |████████████████████████████████| 3.2MB 35.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=5ca97d9461b9b58fa387d6e9ab42e72a3de63fd815ec525163106d77dedfd0bd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
# 載入相關套件, 第一次執行前需安裝 transformers 套件
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re, warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 載入訓練與測試資料
dir = '/content/drive/My Drive/data/ch30'
df = pd.read_csv(dir + '/train.csv')
df_test = pd.read_csv(dir + '/test.csv') 

In [5]:
# 前處理-1 : 消除連字
def decontracted(text):
    # 特殊連字
    text = re.sub(r"(W|w)on(\'|\’)t ", "will not ", text)
    text = re.sub(r"(C|c)an(\'|\’)t ", "can not ", text)
    text = re.sub(r"(Y|y)(\'|\’)all ", "you all ", text)
    text = re.sub(r"(Y|y)a(\'|\’)ll ", "you all ", text)
    # 一般性連字
    text = re.sub(r"(I|i)(\'|\’)m ", "i am ", text)
    text = re.sub(r"(A|a)in(\'|\’)t ", "is not ", text)
    text = re.sub(r"n(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)re ", " are ", text)
    text = re.sub(r"(\'|\’)s ", " is ", text)
    text = re.sub(r"(\'|\’)d ", " would ", text)
    text = re.sub(r"(\'|\’)ll ", " will ", text)
    text = re.sub(r"(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)ve ", " have ", text)
    return text

df['text'] = df['text'].apply(lambda x: decontracted(x))
df_test['text'] = df_test['text'].apply(lambda x: decontracted(x))

In [6]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
# 前處理-2 : 清除特殊符號
import string
regular_punct = list(string.punctuation)
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤']
# 消除標點符號以及上列符號
all_punct = list(set(regular_punct + extra_punct))
# 消除連字號 "-" 以及句號 "."
all_punct.remove('-')
all_punct.remove('.')

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text

df['text'] = df['text'].apply(lambda x: spacing_punctuation(x))
df_test['text'] = df_test['text'].apply(lambda x: spacing_punctuation(x))

In [8]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13 , 000 people receive # wildfires evacuatio...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@ aria _ ahrary @ TheTawniest The out of con...,1
7610,10871,NaN,NaN,M1.94 [ 01 : 04 UTC ] ? 5km S of Volcano Haw...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [9]:
# 前處理-3 : 錯漏字修正
mis_connect_list = ['(W|w)hat', '(W|w)hy', '(H|h)ow', '(W|w)hich', '(W|w)here', '(W|w)ill']
mis_connect_re = re.compile('(%s)' % '|'.join(mis_connect_list))

mis_spell_mapping = {'whattsup': 'WhatsApp', 'whatasapp':'WhatsApp', 'whatsupp':'WhatsApp', 
                      'whatcus':'what cause', 'arewhatsapp': 'are WhatsApp', 'Hwhat':'what',
                      'Whwhat': 'What', 'whatshapp':'WhatsApp', 'howhat':'how that',
                      # why
                      'Whybis':'Why is', 'laowhy86':'Foreigners who do not respect China',
                      'Whyco-education':'Why co-education',
                      # How
                      "Howddo":"How do", 'Howeber':'However', 'Showh':'Show',
                      "Willowmagic":'Willow magic', 'WillsEye':'Will Eye', 'Williby':'will by'}
def spacing_some_connect_words(text):
    """
    'Whyare' -> 'Why are'
    """
    ori = text
    for error in mis_spell_mapping:
        if error in text:
            text = text.replace(error, mis_spell_mapping[error])
            
    # what
    text = re.sub(r" (W|w)hat+(s)*[A|a]*(p)+ ", " WhatsApp ", text)
    text = re.sub(r" (W|w)hat\S ", " What ", text)
    text = re.sub(r" \S(W|w)hat ", " What ", text)
    # why
    text = re.sub(r" (W|w)hy\S ", " Why ", text)
    text = re.sub(r" \S(W|w)hy ", " Why ", text)
    # How
    text = re.sub(r" (H|h)ow\S ", " How ", text)
    text = re.sub(r" \S(H|h)ow ", " How ", text)
    # which
    text = re.sub(r" (W|w)hich\S ", " Which ", text)
    text = re.sub(r" \S(W|w)hich ", " Which ", text)
    # where
    text = re.sub(r" (W|w)here\S ", " Where ", text)
    text = re.sub(r" \S(W|w)here ", " Where ", text)
    
    text = mis_connect_re.sub(r" \1 ", text)
    text = text.replace("What sApp", 'WhatsApp') 
    return text

df['text'] = df['text'].apply(lambda x: spacing_some_connect_words(x))
df_test['text'] = df_test['text'].apply(lambda x: spacing_some_connect_words(x))

In [10]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13 , 000 people receive # wildfires evacuatio...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@ aria _ ahrary @ TheTawniest The out of con...,1
7610,10871,NaN,NaN,M1.94 [ 01 : 04 UTC ] ? 5km S of Volcano Haw...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [11]:
# 載入 distilBERT 模型或 Bert 模型 (下列兩行中, 將不選的模型註解掉即可)
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# 載入預訓練權重以及 tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [12]:
# 調整訓練資料的大小 (可取消, 若不取消表示取前4000筆訓練)
df = df[:4000]

In [13]:
# 將訓練資料經過 tokenizer 編碼轉換
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [14]:
tokenized

0       [101, 2256, 15616, 2024, 1996, 3114, 1997, 202...
1       [101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...
2       [101, 2035, 3901, 2356, 2000, 1005, 7713, 1999...
3       [101, 2410, 1010, 2199, 2111, 4374, 1001, 3748...
4       [101, 2074, 2288, 2741, 2023, 6302, 2013, 1009...
                              ...                        
3995    [101, 2031, 2017, 2412, 4622, 2019, 2214, 2299...
3996    [101, 14295, 5645, 2633, 9609, 1004, 23713, 10...
3997    [101, 1001, 7186, 1001, 7071, 8191, 14295, 102...
3998    [101, 6353, 2757, 2349, 2000, 14295, 1999, 126...
3999    [101, 15393, 2061, 12672, 10626, 8107, 2044, 1...
Name: text, Length: 4000, dtype: object

In [15]:
# 以最長字串為準, 將訓練資料補零成相同長度
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [16]:
# 設定 attention_mask, 將計算經過 Bert 生成的 Embedding 結果, 儲存於 last_hidden_states 中
attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded).to(torch.int64)
attention_mask = torch.tensor(attention_mask).to(torch.int64)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [17]:
# 準備下一階段要用的特徵 (上階段 Embedding 結果) 與目標值
labels = df['target']
features = last_hidden_states[0][:,0,:].numpy()
features[0].shape

(768,)

In [18]:
# 切割訓練 / 測試集
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [19]:
# 對 Logistic Regression 跑參, 相當於加上單層類神經網路
import sklearn
from sklearn.model_selection import GridSearchCV
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)
print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.7903333333333333


In [20]:
# 將上一格跑出的 Logistic Regression 最佳 C 值填入, 觀察測試集的驗證分數
lr_clf = LogisticRegression(C = 5.263252631578947)  
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

0.783

In [22]:
# 將預測目標資料經過 tokenizer 編碼轉換
tokenized_t = df_test['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [23]:
# 以最長字串為準, 將預測目標資料補零成相同長度
max_len = 0
for i in tokenized_t.values:
    if len(i) > max_len:
        max_len = len(i)
        
padded_t = np.array([i + [0]*(max_len-len(i)) for i in tokenized_t.values])
np.array(padded_t).shape

(3263, 73)

In [24]:
# 設定 attention_mask, 將計算經過 Bert 生成的 Embedding 結果, 儲存於 last_hidden_states 中
attention_mask_t = np.where(padded_t != 0, 1, 0)
input_ids = torch.tensor(padded_t).to(torch.int64)
attention_mask_t = torch.tensor(attention_mask_t).to(torch.int64)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask_t)

In [25]:
# 輸出預測目標資料的預測結果
val_features = last_hidden_states[0][:,0,:].numpy() 
y_pred = lr_clf.predict(val_features)
y_pred

array([1, 1, 1, ..., 1, 1, 0])